In [39]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from dataclasses import dataclass
import matplotlib.pyplot as plt
from datetime import timedelta

In [25]:
os.getcwd()

'/Users/Christian/Desktop/MSMFT/Semester 2/Fixed Income/MF728-RiskAnalysisProject/Main Coding File'

In [26]:
data = pd.read_csv("../Data/CouponBonds/CouponBondData/YieldData/A Yield.csv", index_col = 0,header = list(range(8)))

data.sort_index(inplace = True)
data.index = pd.to_datetime(data.index)

data = data.loc["2020-01-01":"2020-12-31"]

In [ ]:
class CIR_base:
    """
    CIR model class that takes in a series of yields.
    """

    def __init__(self, rates: pd.Series):
        """
        Initialize the CIR model with a series of yields.
        
        Parameters
        ----------
        rates : pd.DataFrame
            A DataFrame containing the yield data.
        """
        self.rates = rates

    def calibrate(self):
        """Calibrate CIR model parameters using OLS regression."""
        self.rates, rates = self.rates/100, self.rates/100

        rt = rates[1:]

        #setting up the parameters for OLS regression
        delta_rt = rates.diff().dropna()
        sqrt_rt = np.sqrt(rt)
        delta_t = pd.Series(rates.index.to_series().diff().dropna().dt.days.values, index=delta_rt.index) / 360 * 12  # Ensure matching lengths
        T = (rates.index[-1] - rates.index[0]).days / 360 * 12  # Total time period in years

        #setting up the OLS regression
        y = delta_rt/sqrt_rt
        Y = np.array(y).reshape(-1, 1)

        x1 = delta_t/sqrt_rt
        x2 = sqrt_rt*delta_t

        X = np.array([x1, x2]).T

        # Fit the linear regression model
        model = LinearRegression(fit_intercept=False)
        model.fit(X, Y)

        # Extract the coefficients
        a, b = model.coef_[0]

        # Calculate the parameters
        kappa = -b
        theta = a/kappa
        sigma = np.sqrt(np.square(delta_rt/sqrt_rt).sum() / T)
        
        self.model = model
        self.theta = theta
        self.kappa = kappa
        self.sigma = sigma

    def simulate(self, N:int = 1, starting_rate:float = None) ->np.ndarray:
        """
        Simulate the CIR process using calibrated parameters.
        
        Parameters
        ----------
        N : int
            Number of simulations to run.
        starting_rate : float
            Starting interest rate for the simulation. If None, uses the first rate in the series.

        Returns
        -------
        np.ndarray
            Simulated interest rates. Each row corresponds to a simulation, and each column corresponds to a time step.
        """
        # Calibrate the CIR process
        self.calibrate()

        if starting_rate is None:
            starting_rate = self.rates.iloc[0]

        # Getting CIR parameters
        theta = self.theta
        kappa = self.kappa
        sigma = self.sigma
        delta_t = 1/360

        rates = np.zeros((N, len(self.rates)))

        for i in range(N):
            # Simulate the process
            # Use the CIR model parameters to generate future rates


            Z = np.random.normal(0, 1, size=(len(self.rates)))

            rates[i, :] = np.zeros(len(self.rates))
            rates[i, 0] = starting_rate

            for t in range(1, len(self.rates)):
                rates[i, t] = rates[i,t-1] + kappa * (theta - rates[i,t-1]) * delta_t + sigma * np.sqrt(delta_t) * np.sqrt(rates[i,t-1]) * Z[t-1]

        return rates
    
class CIR(CIR_base):
    """
    CIR model class that takes in a series of yields.
    """

    def __init__(self, rates: pd.DataFrame):
        """
        Initialize the CIR model with a series of yields.
        
        Parameters
        ----------
        rates : pd.DataFrame
            A DataFrame containing the yield data.
        """

        self.rates = rates/100

        
        self.dataframe = rates.apply(lambda x : CIR_base(x), axis=0)

    def calibrate(self):
        self.dataframe.apply(lambda x : x.calibrate())
    
    def simulate(self, starting_rates:pd.Series|None = None) ->np.ndarray:
        """
        Simulate the CIR process using calibrated parameters.
        
        Parameters
        ----------
        N : int
            Number of simulations to run.
        starting_rate : float
            Starting interest rate for the simulation. If None, uses the first rate in the series.

        Returns
        -------
        np.ndarray
            Simulated interest rates. Each row corresponds to a simulation, and each column corresponds to a time step.
        """
        # Calibrate the CIR process

        simulated_rates = self.dataframe.apply(lambda x : pd.Series(x.simulate()[0])).T

        simulated_rates.index = [self.rates.index[0] + timedelta(days = i) for i in range(len(simulated_rates))] 

        return simulated_rates
    

model = CIR(data)
model.calibrate()

simulations = model.simulate()

simulations


341099CH0 Corp      92976GAJ0 Corp      983024AN0 Corp  \
Company       Duke Energy Florida LLC Wells Fargo Bank NA           Wyeth LLC   
Ticker                            DUK                 WFC                 PFE   
Rating                              A                   A                   A   
Issue Date        2007-09-18 00:00:00 2007-12-07 00:00:00 2007-03-27 00:00:00   
Maturity Date     2037-09-15 00:00:00 2038-01-15 00:00:00 2037-04-01 00:00:00   
Coupon                           6.35                 6.6                5.95   
Tenor (days)                    10955               10997               10963   
2020-01-02                   0.001027            0.001027            0.001027   
2020-01-03                   0.001013            0.001026            0.001015   
2020-01-04                   0.001018            0.001019            0.001027   
2020-01-05                   0.001001            0.001012            0.001020   
2020-01-06                   0.001005            0.001044            0.001050   
...                               ...                 ...                 ...   
2020-09-04                   0.000662            0.000886            0.001439   
2020-09-05                   0.000642            0.000901            0.001450   
2020-09-06                   0.000640            0.000913            0.001456   
2020-09-07                   0.000664            0.000911            0.001422   
2020-09-08                   0.000623            0.000923            0.001421   

                                  74456QAR7 Corp  \
Company       Public Service Electric and Gas Co   
Ticker                                       PEG   
Rating                                         A   
Issue Date                   2007-05-14 00:00:00   
Maturity Date                2037-05-01 00:00:00   
Coupon                                       5.8   
Tenor (days)                               10945   
2020-01-02                              0.001027   
2020-01-03                              0.000999   
2020-01-04                              0.001030   
2020-01-05                              0.001025   
2020-01-06                              0.001032   
...                                          ...   
2020-09-04                              0.000903   
2020-09-05                              0.000908   
2020-09-06                              0.000901   
2020-09-07                              0.000869   
2020-09-08                              0.000882   

                                  207597DX0 Corp  \
Company       Connecticut Light and Power Co/The   
Ticker                                        ES   
Rating                                         A   
Issue Date                   2007-03-27 00:00:00   
Maturity Date                2037-03-01 00:00:00   
Coupon                                      5.75   
Tenor (days)                               10932   
2020-01-02                              0.001027   
2020-01-03                              0.001021   
2020-01-04                              0.001001   
2020-01-05                              0.000997   
2020-01-06                              0.001013   
...                                          ...   
2020-09-04                              0.000988   
2020-09-05                              0.001002   
2020-09-06                              0.001036   
2020-09-07                              0.001046   
2020-09-08                              0.001050   

                               771367BZ1 Corp      695114CG1 Corp  \
Company       Rochester Gas and Electric Corp          PacifiCorp   
Ticker                                    AGR              BRKHEC   
Rating                                      A                   A   
Issue Date                2007-07-17 00:00:00 2007-10-03 00:00:00   
Maturity Date             2032-07-15 00:00:00 2037-10-15 00:00:00   
Coupon                                   6.47                6.25   
Tenor (days)          

In [28]:
data.loc["2020-01-01":"2020-12-31"].iloc[:,2].plot()

<Axes: >